**Audio Restoration for Generative Models — Improving MusicGen Outputs**

*Giada Manfredi*

January 2026

In [ ]:
!pip install noisereduce
!pip install pedalboard
!pip install librosa
!pip install demucs

In [2]:
import librosa
import librosa.display
import numpy as np
from google.colab import files
import os
from pedalboard.io import AudioFile
from pedalboard import *
import noisereduce as nr
import soundfile as sf
from IPython.display import Audio, display
import matplotlib.pyplot as plt
from demucs import pretrained
from demucs.apply import apply_model
import torch

In [ ]:
# 1. CARICAMENTO AUDIO

# Messaggio iniziale per invitare l'utente a caricare un file audio
print("Carica il tuo file audio.")

# Caricamento del file tramite l'interfaccia di upload di Colab
uploaded = files.upload()

if uploaded:
    for fn in uploaded.keys():
        audio_file_path = f"/content/{fn}"
        print(f"File '{fn}' caricato.")

        try:
            # Carica il file audio con librosa
            # sr=None significa che viene mantenuta la frequenza di campionamento originale
            # y contiene i dati audio, sr contiene la frequenza di campionamento
            y, sr = librosa.load(audio_file_path, sr=None)

            print(f"File audio caricato con successo: {audio_file_path}")
            print(f"Frequenza di campionamento (sr): {sr} Hz")

        except FileNotFoundError:
            print(f"Errore: Il file '{audio_file_path}' non è stato trovato. Controlla il percorso.")
        except Exception as e:
            print(f"Si è verificato un errore durante il caricamento del file audio: {e}")
else:
    print("Nessun file selezionato.")

In [ ]:
# 2. RESTAURAZIONE AUDIO DIRETTA

# Frequenza di campionamento target
target_sr = 44100  # Hz

# Controlla se è necessario ricampionare l'audio
if sr != target_sr:
    print(f"Ricampionamento dell'audio da {sr} Hz a {target_sr} Hz.")
    audio_to_process = librosa.resample(y=y, orig_sr=sr, target_sr=target_sr)
    current_sr = target_sr
else:
    audio_to_process = y
    current_sr = sr

# Riduzione del rumore
# stationary=True indica che il rumore è costante nel tempo
# prop_decrease=0.75 controlla di quanto ridurre il rumore
reduced_noise = nr.reduce_noise(y=audio_to_process, sr=current_sr, stationary=True, prop_decrease=0.75)

# Creazione della catena di effetti tramite Pedalboard
# NoiseGate: elimina i suoni al di sotto di una certa soglia
# Compressor: uniforma il volume dell'audio
# LowShelfFilter: aumenta le frequenze basse (bassi)
# Gain: aumenta il volume complessivo
board = Pedalboard([
    NoiseGate(threshold_db=-30, ratio=1.5, release_ms=250),
    Compressor(threshold_db=-16, ratio=4),
    LowShelfFilter(cutoff_frequency_hz=400, gain_db=10, q=1),
    Gain(gain_db=2)
])

# Applica gli effetti all'audio ridotto dal rumore
effected_audio = board(reduced_noise, current_sr)

# Salvataggio dell'audio restaurato
num_channels0 = 1 if len(effected_audio.shape) == 1 else effected_audio.shape[0]
output_filename_pedalboard = '/content/audio_enhenced.wav'

with AudioFile(output_filename_pedalboard, 'w', current_sr, num_channels=num_channels0) as f:
    f.write(effected_audio)

print(f"Audio elaborato e salvato come '{output_filename_pedalboard}'.")

In [ ]:
# 3. RESTAURAZIONE AUDIO STEM-WISE

print("Caricamento modello Demucs...")
# Carica il modello pre-addestrato per la separazione delle tracce
model = pretrained.get_model('mdx')

# Per MDX è necessario un audio stereo
# Convertiamo l'audio in tensor PyTorch
audio_tensor = torch.tensor(audio_to_process, dtype=torch.float32)
audio_tensor = audio_tensor.unsqueeze(0).unsqueeze(0)  # Shape: (1,1,N)
audio_tensor = audio_tensor.repeat(1, 2, 1)            # Shape: (1,2,N), duplicando i canali per stereo

print("Separazione in tracce...")
with torch.no_grad():
    # Applica il modello Demucs
    # shifts: 0, split: True, overlap: 0.0, transition_power: 1.0
    # Output atteso: (batch, num_sources, num_channels, num_samples)
    demucs_output = apply_model(
        model,
        audio_tensor,
        0,      # shifts
        True,   # split
        0.0,    # overlap
        1.0     # transition_power
    )

# Rimuove le dimensioni di lunghezza 1 (batch)
# Risultato atteso: (num_sources, num_channels, num_samples)
separated_sources_processed = demucs_output.squeeze()

# Crea la cartella per salvare le tracce separate
output_dir = "/content/separated_tracks"
os.makedirs(output_dir, exist_ok=True)

# Salva ogni traccia separata
for i, name in enumerate(model.sources):
    src_tensor = separated_sources_processed[i]  # (num_channels, num_samples)

    # Controllo aggiuntivo per eventuali dimensioni extra inattese
    if src_tensor.ndim > 2:
        src_tensor = src_tensor[0]  # Prende la prima dimensione extra

    # Trasponi per soundfile.write: (num_samples, num_channels)
    src = src_tensor.cpu().numpy().T
    sf.write(os.path.join(output_dir, f"{name}.wav"), src, current_sr)

print("Separazione completata.")

# Cartelle per le tracce pulite
input_dir = "/content/separated_tracks"
output_dir = "/content/cleaned_separated_tracks"
os.makedirs(output_dir, exist_ok=True)

cleaned_tracks_data = []

# Lista dei file audio separati
audio_files = [f for f in os.listdir(input_dir) if f.endswith('.wav')]

# Ciclo di pulizia e restauro di ogni traccia
for filename in audio_files:
    track_path = os.path.join(input_dir, filename)
    track_name = os.path.splitext(filename)[0]

    print(f"Processando: {track_name}")

    # Carica la traccia
    y_track, sr_track = librosa.load(track_path, sr=current_sr, mono=False)

    # Se stereo, converte in mono mediando i canali
    if y_track.ndim > 1:
        if y_track.shape[0] == 2:
            y_track_mono = y_track.mean(axis=0)
        else:
            y_track_mono = y_track[0]
    else:
        y_track_mono = y_track

    # Riduzione del rumore
    reduced_noise_track = nr.reduce_noise(
        y=y_track_mono,
        sr=sr_track,
        stationary=True,
        prop_decrease=0.75
    )

    # Applicazione della catena di effetti
    board = Pedalboard([
        NoiseGate(threshold_db=-30, ratio=1.5, release_ms=250),
        Compressor(threshold_db=-16, ratio=4),
        LowShelfFilter(cutoff_frequency_hz=400, gain_db=10, q=1),
        Gain(gain_db=2)
    ])

    effected_audio_track = board(reduced_noise_track, sr_track)

    # Salva i dati della traccia pulita in memoria
    cleaned_tracks_data.append({
        'name': track_name,
        'audio': effected_audio_track,
        'sr': sr_track
    })
    print(f"Finito di processare {track_name}.")

print("Tutte le tracce sono restaurate.")

# Salvataggio delle tracce restaurate
print("Salvando le tracce restaurate...")
for track_data in cleaned_tracks_data:
    track_name = track_data['name']
    audio_data = track_data['audio']
    sample_rate = track_data['sr']

    output_filepath = os.path.join(output_dir, f"{track_name}_cleaned.wav")

    with AudioFile(output_filepath, 'w', sample_rate, num_channels=1) as f:
        f.write(audio_data)

    print(f"Traccia restaurata salvata: {output_filepath}")

print("Tutte le tracce restaurate sono state salvate.")

In [ ]:
# 4. CARICAMENTO E RIPRODUZIONE DELLE TARCCE ORIGINALI E RESTAURATE

original_tracks_data = []

# Cartella delle tracce originali separate (creata nel passo precedente)
input_dir_original = "/content/separated_tracks"

# Verifica che current_sr sia definito
if 'current_sr' not in locals():
    print("Attenzione: current_sr non trovato. Impostato a 44100 Hz di default.")
    current_sr = 44100

# Caricamento delle tracce originali
for filename in os.listdir(input_dir_original):
    if filename.endswith('.wav'):
        track_path = os.path.join(input_dir_original, filename)
        track_name = os.path.splitext(filename)[0]

        # Carica la traccia senza ricampionamento (stereo se presente)
        y_track, sr_track = librosa.load(track_path, sr=current_sr, mono=False)

        # Se la traccia è stereo, converte in mono per una riproduzione coerente
        if y_track.ndim > 1 and y_track.shape[0] == 2:
            y_track_mono = y_track.mean(axis=0)
        else:
            y_track_mono = y_track  # già mono o canale singolo

        # Aggiunge la traccia ai dati originali
        original_tracks_data.append({
            'name': track_name,
            'audio': y_track_mono,
            'sr': sr_track
        })

# Riproduzione delle tracce originali
print("Riproduzione delle tracce originali:")
for track_data in original_tracks_data:
    track_name = track_data['name']
    audio_data = track_data['audio']
    sample_rate = track_data['sr']
    print(f"Riproduzione della traccia originale: {track_name}")
    display(Audio(audio_data, rate=sample_rate))

# Riproduzione delle tracce restaurate
print("Riproduzione delle tracce restaurate:")
for track_data in cleaned_tracks_data:
    track_name = track_data['name']
    audio_data = track_data['audio']
    sample_rate = track_data['sr']
    print(f"Riproduzione della traccia restaurata: {track_name}")
    display(Audio(audio_data, rate=sample_rate))



In [ ]:
# 5. CONFRONTO DEGLI SPETTROGRAMMI DELLE TRACCE ORIGINALI E RESTAURATE

print("Confronto fra gli spettrogrammi delle tracce originali e restaurate")

all_spectrograms_db_data = []  # Memorizza tutti gli spettrogrammi in dB
min_db_values = []  # Per calcolare il minimo globale (scala colore uniforme)
max_db_values = []  # Per calcolare il massimo globale (scala colore uniforme)

# -------------------------
# Elaborazione delle tracce originali (ora chiamate input)
# -------------------------
for track_data in original_tracks_data:
    y_track = track_data['audio']
    sr_track = track_data['sr']
    track_name = track_data['name']

    # Calcola lo spettrogramma di Mel
    S = librosa.feature.melspectrogram(y=y_track, sr=sr_track)
    S_db = librosa.power_to_db(S, ref=np.max)  # Converte in decibel

    # Etichetta: nome traccia fra virgolette + tipo "input" alla fine
    display_name = f'"{track_name}" input'

    all_spectrograms_db_data.append({
        'name': display_name,
        'spectrogram_db': S_db,
        'sr': sr_track
    })

    min_db_values.append(S_db.min())
    max_db_values.append(S_db.max())

# -------------------------
# Elaborazione delle tracce pulite (ora chiamate restored)
# -------------------------
for track_data in cleaned_tracks_data:
    y_track = track_data['audio']
    sr_track = track_data['sr']
    track_name = track_data['name']

    S = librosa.feature.melspectrogram(y=y_track, sr=sr_track)
    S_db = librosa.power_to_db(S, ref=np.max)

    # Etichetta: nome traccia fra virgolette + tipo "restored" alla fine
    display_name = f'"{track_name}" restored'

    all_spectrograms_db_data.append({
        'name': display_name,
        'spectrogram_db': S_db,
        'sr': sr_track
    })

    min_db_values.append(S_db.min())
    max_db_values.append(S_db.max())

# Calcola min e max globali per la scala dei colori uniforme tra tutte le tracce
global_vmin = np.min(min_db_values)
global_vmax = np.max(max_db_values)

# Elenco dei nomi delle tracce senza distinzione input/restored
unique_track_names = [data['name'].replace('"', '').replace(' input', '').replace(' restored', '')
                      for data in all_spectrograms_db_data]
unique_track_names = sorted(list(set(unique_track_names)))

# Inizializza la figura con righe = numero di tracce, 2 colonne (input | restored)
fig, axes = plt.subplots(nrows=len(unique_track_names), ncols=2,
                         figsize=(15, 5 * len(unique_track_names)))

# Gestione caso con una sola riga di subplot
if len(unique_track_names) == 1:
    axes = np.expand_dims(axes, axis=0)

# Plot degli spettrogrammi
for i, track_name in enumerate(unique_track_names):
    # Seleziona i dati input e restored per la traccia corrente
    input_spec_data = next((item for item in all_spectrograms_db_data if item['name'] == f'"{track_name}" input'), None)
    restored_spec_data = next((item for item in all_spectrograms_db_data if item['name'] == f'"{track_name}" restored'), None)

    if input_spec_data and restored_spec_data:
        # -------------------------
        # Spettrogramma input
        # -------------------------
        ax_input = axes[i, 0]
        librosa.display.specshow(input_spec_data['spectrogram_db'], sr=input_spec_data['sr'],
                                 x_axis='time', y_axis='mel', vmin=global_vmin, vmax=global_vmax, ax=ax_input)
        fig.colorbar(ax_input.collections[0], format='%+2.0f dB', ax=ax_input)
        ax_input.set_title(input_spec_data['name'])
        ax_input.set_xlabel('time (s)')
        ax_input.set_ylabel('frequency (Hz)')

        # -------------------------
        # Spettrogramma restored
        # -------------------------
        ax_restored = axes[i, 1]
        librosa.display.specshow(restored_spec_data['spectrogram_db'], sr=restored_spec_data['sr'],
                                 x_axis='time', y_axis='mel', vmin=global_vmin, vmax=global_vmax, ax=ax_restored)
        fig.colorbar(ax_restored.collections[0], format='%+2.0f dB', ax=ax_restored)
        ax_restored.set_title(restored_spec_data['name'])
        ax_restored.set_xlabel('time (s)')
        ax_restored.set_ylabel('frequency (Hz)')
    else:
        print(f"Warning: Dati spettrogramma non trovati per la traccia {track_name}")

# Ottimizza il layout e mostra la figura
plt.tight_layout()
plt.show()

In [ ]:
# 6. RICOSTRUZIONE DELL'AUDIO DA TUTTE LE TRACCE PULITE

# Estrai tutte le tracce audio pulite
all_cleaned_audio_data = [track['audio'] for track in cleaned_tracks_data]

# Trova la lunghezza massima tra tutte le tracce
max_length = max(len(audio) for audio in all_cleaned_audio_data)

# Inizializza un array di zeri con la lunghezza massima e tipo float32
reconstructed_audio = np.zeros(max_length, dtype=np.float32)

# Itera attraverso tutte le tracce pulite, effettua il padding e somma
for audio_track in all_cleaned_audio_data:
    # Aggiungi zeri alla fine della traccia per farla arrivare a max_length
    padded_track = np.pad(audio_track, (0, max_length - len(audio_track)), 'constant')
    # Somma la traccia al mix ricostruito
    reconstructed_audio += padded_track

print(f"Audio ricomposto creato con successo.")
print(f"Durata: {len(reconstructed_audio)} campioni ({len(reconstructed_audio)/current_sr:.2f} secondi).")
print(f"Frequenza di campionamento utilizzata: {current_sr} Hz.")

# -------------------------
# Salva l'audio ricomposto in un file WAV
# -------------------------
output_reconstructed_file = '/content/reconstructed_audio.wav'

# Se vuoi usare Pedalboard o sf.write, qui usiamo soundfile (sf)
import soundfile as sf
sf.write(output_reconstructed_file, reconstructed_audio, current_sr)

print(f"Audio ricomposto salvato come: {output_reconstructed_file}")

In [ ]:
#7. RIPRODUZIONE DELLA VERSIONE ORIGINALE E DELLE RESTAURATE

# Audio originale
print("Riproduzione dell'audio originale:")
print(f"Durata: {len(y)/sr:.2f} secondi, Frequenza di campionamento: {sr} Hz")
display(Audio(y, rate=sr))

# Audio migliorato senza separazione in tracce
print("Riproduzione dell'audio migliorato (senza separazione in tracce):")
print(f"Durata: {len(effected_audio)/current_sr:.2f} secondi, Frequenza di campionamento: {current_sr} Hz")
display(Audio(effected_audio, rate=current_sr))

# Audio ricomposto sommando tutte le tracce pulite
print("Riproduzione dell'audio migliorato dividendo in tracce e ricomponendo:")
print(f"Durata: {len(reconstructed_audio)/current_sr:.2f} secondi, Frequenza di campionamento: {current_sr} Hz")
display(Audio(reconstructed_audio, rate=current_sr))

In [ ]:
#8. SPETTROGRAMMI

# Assumendo che y1 = y sia l'audio originale
y1 = y

# Calcola gli spettrogrammi Mel
S_input = librosa.feature.melspectrogram(y=y1, sr=current_sr)
S_direct = librosa.feature.melspectrogram(y=effected_audio, sr=current_sr)
S_stemwise = librosa.feature.melspectrogram(y=reconstructed_audio, sr=current_sr)  # <-- stem-wise restoration

# Converti in decibel
S_db_input = librosa.power_to_db(S_input, ref=np.max)
S_db_direct = librosa.power_to_db(S_direct, ref=np.max)
S_db_stemwise = librosa.power_to_db(S_stemwise, ref=np.max)

# Trova min e max globali per la scala colori uniforme
vmin = min(S_db_input.min(), S_db_direct.min(), S_db_stemwise.min())
vmax = max(S_db_input.max(), S_db_direct.max(), S_db_stemwise.max())

# Crea la figura con 3 subplot
plt.figure(figsize=(18, 6))

# Subplot 1: input audio
plt.subplot(1, 3, 1)
librosa.display.specshow(S_db_input, sr=current_sr, x_axis='time', y_axis='mel', vmin=vmin, vmax=vmax)
plt.colorbar(format='%+2.0f dB')
plt.title('input audio')
plt.xlabel('time (s)')
plt.ylabel('frequency (Hz)')

# Subplot 2: direct restoration
plt.subplot(1, 3, 2)
librosa.display.specshow(S_db_direct, sr=current_sr, x_axis='time', y_axis='mel', vmin=vmin, vmax=vmax)
plt.colorbar(format='%+2.0f dB')
plt.title('direct restoration')
plt.xlabel('time (s)')
plt.ylabel('frequency (Hz)')

# Subplot 3: stem-wise restoration
plt.subplot(1, 3, 3)
librosa.display.specshow(S_db_stemwise, sr=current_sr, x_axis='time', y_axis='mel', vmin=vmin, vmax=vmax)
plt.colorbar(format='%+2.0f dB')
plt.title('stem-wise restoration')
plt.xlabel('time (s)')
plt.ylabel('frequency (Hz)')

plt.tight_layout()
plt.show()

print("Spettrogrammi generati e visualizzati per input audio, direct restoration e stem-wise restoration con la stessa scala di colori.")


In [ ]:
# 9. ANALISI DEL 25° PERCENTILE DEGLI SPETTROGRAMMI

# Calcola il 25° percentile (energia a bassa ampiezza) per ciascun spettrogramma

# Spettrogramma originale
percentile_25_original = np.percentile(S_db_input, 25)

# Spettrogramma migliorato senza dividere in tracce (direct restoration)
percentile_25_direct = np.percentile(S_db_direct, 25)

# Spettrogramma migliorato dividendo in tracce (hybrid restoration)
percentile_25_stemwise = np.percentile(S_db_stemwise, 25)

# Stampa i valori calcolati
print(f"25° percentile dello spettrogramma input audio: {percentile_25_original:.2f} dB")
print(f"25° percentile dello spettrogramma direct restoration: {percentile_25_direct:.2f} dB")
print(f"25° percentile dello spettrogramma stem-wise restoration: {percentile_25_stemwise:.2f} dB\n")

# Confronto per determinare quale metodo riduce maggiormente il rumore a bassa ampiezza
if percentile_25_stemwise < percentile_25_original and percentile_25_stemwise < percentile_25_direct:
    print("Il 25° percentile è più basso con la stem-wise restoration, quindi é questo il caso in cui si ha una maggiore riduzione del rumore.")
elif percentile_25_direct < percentile_25_original and percentile_25_direct < percentile_25_stemwise:
    print("Il 25° percentile è più basso con la direct restoration, quindi é questo il caso in cui si ha una  maggiore riduzione del rumore.")
else:
    print("Il 25° percentile più basso è nell'audio input: le versioni migliorate non hanno ridotto il rumore a bassa ampiezza.")